In [35]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDRegressor

In [37]:
original_df = pd.read_csv('CreditDataset_with_missing_income.csv')
categorial_feature_names=['Home', 'Marital','Job']
numeric_features_names = ['Seniority', 'Time', 'Age', 'Records', 'Expenses', 'Income', 'Assets', 'Debt', 'Amount', 'Price']
missing_values = {'Income': [0,99999999], 'Assets': [99999999], 'Debt': [99999999], 'Home': [0]}
target = 'Income'
estimator_dict = {'Home': DecisionTreeClassifier(max_depth= 7, criterion = 'entropy'), 'Income': SGDRegressor(loss = 'squared_loss')}

In [38]:
from prepareData1 import separateTrainAndMissedDF

In [39]:
final_train_df,final_predict_df, target_df, df_with_missing_target = separateTrainAndMissedDF(original_df, categorial_feature_names, numeric_features_names, missing_values, target)

C:\Users\Dmitry_Nikiforov\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [40]:
df_with_missing_target

,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Assets,Debt,Amount,Price,Source
0,9,1,60,30,2,1,3,73,0,0,800,846,Calc
1,17,1,60,58,3,1,1,48,0,0,1000,1658,Calc
2,10,2,36,46,2,2,3,90,3000,0,2000,2985,Calc
3,0,1,60,24,1,1,1,63,2500,0,900,1325,Calc
4,0,1,36,26,1,1,1,46,0,0,310,910,Calc


In [41]:
print("Размер датасета для обучения {}".format(final_train_df.shape))
print("Размер датасета для заполнения {}".format(final_predict_df.shape))

Размер датасета для обучения (4035, 25)
Размер датасета для заполнения (5, 25)


In [42]:
def predict_and_get_enhance_data(train_x, train_y, test_prepare_x, test_x, target, estimator_dict):
    estimator = estimator_dict[target]
    estimator.fit(train_x.values, train_y.values)
    predicted_target = pd.Series(data = estimator.predict(test_prepare_x), index = test_prepare_x.index).to_frame(target)
    predicted_data = pd.merge(test_x, predicted_target, how='left', left_index=True, right_index=True)
    return predicted_data

In [43]:
predict_and_get_enhance_data(final_train_df, target_df, final_predict_df, df_with_missing_target, target, estimator_dict)

C:\Users\Dmitry_Nikiforov\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Assets,Debt,Amount,Price,Source,Income
0,9,1,60,30,2,1,3,73,0,0,800,846,Calc,124.719030
1,17,1,60,58,3,1,1,48,0,0,1000,1658,Calc,115.993023
2,10,2,36,46,2,2,3,90,3000,0,2000,2985,Calc,231.859416
3,0,1,60,24,1,1,1,63,2500,0,900,1325,Calc,131.894899
4,0,1,36,26,1,1,1,46,0,0,310,910,Calc,106.572709
